# <center> Mothur tutorial <br/> Sequence Analysis</center>
I will use **Mothur** to figure out what OUTs are present in a fecal sample of a mice and a Mock sample. This does not take too long, has a limited memory footprint but can be expanded to higher number of samples. I base this off of the Tutorial that can be found at https://www.mothur.org/wiki/MiSeq_SOP [1]. I diverge from it at the choice of data, as well as I will use the up to date reference files.

# Read data processing
**Mothur**  is a full data analysis suite, so it can attach to your research pipeline at the point where raw reads are produced.

** \*\* Note that you need to have the Mothur() Bash function (see Installation/Usage part) set up to run this tutorial! **

## 1\. Contig assembly

First we make **Mothur** discover pairended read files in the `Data` folder.

In [12]:
Mothur "make.file(inputdir=Data, type=gz, prefix=tutor)"

mothur > make.file(inputdir=Data, type=gz, prefix=tutor)
Setting input directory to: Data/
Output File Names: 
Data/tutor.single.files
Data/tutor.paired.files


Whenever a step is completed **Mothur** will generate the files instead of variables to save operative memory. The processes are largely sequential and generate a lot of files with **very** long names. There is no control over how these files are named after a function completes. Generally they make sense, but it is relatively difficult to keep track of them, this is the main reason why I elected to use Jupyter with a Bash kernel to do this tutorial. - It helps to keep together my workflow and the commands I use.

It was greedy produced the pair ended reads `.fastq.gz` and the `HMP_MOCK.v35.fasta.gz` which is not a suitable file for contig assembly. **Mothur** believes this a sample as well, but since it is not a pairended FASTA file, we can fix this easily. We only want to keep the dictionary of samples which have paired read files. Over time the names of the files downstream in our process will have more and more additions separated by dots, so let's also rename the file we need to keep it simple. Otherwise **Mothur** will carry the `.parired` ending.

In [13]:
rm Data/tutor.single.files
mv Data/tutor.paired.files Data/tutor.files

Now ontu the real contig assembly, it is a single command. There ar options like `processors=2` that is worth using, I will not talk about fine tuning this command.

In [16]:
Mothur "make.contigs(file=Data/tutor.files, processors=2)"

mothur > make.contigs(file=Data/tutor.files, processors=2)
Using 2 processors.
>>>>>	Processing file pair Data/Mock_S280_L001_R1_001.fastq.gz - Data/Mock_S280_L001_R2_001.fastq.gz (files 2 of 2)	<<<<<
Making contigs...
1000
2000
3000
4000
4779
Done.
It took 4 secs to assemble 4779 reads.
mothur > set.logfile(name=mothur.log, append=T)
mothur > make.contigs(file=Data/tutor.files, processors=2)
Using 2 processors.
>>>>>	Processing file pair Data/F3D150_S216_L001_R1_001.fastq.gz - Data/F3D150_S216_L001_R2_001.fastq.gz (files 1 of 2)	<<<<<
Making contigs...
1000
2000
3000
4000
5000
5509
Done.
It took 5 secs to assemble 5509 reads.
It took 5 secs to process 10288 sequences.
Group count: 
F3D150	5509
Mock	4779
Total of all groups is 10288
Output File Names: 
Data/tutor.trim.contigs.fasta
Data/tutor.trim.contigs.qual
Data/tutor.contigs.report
Data/tutor.scrap.contigs.fasta
Data/tutor.scrap.contigs.qual
Data/tutor.contigs.groups
[WARNING]: your sequence names contained ':'.  I changed them to 

**Mothur** is very verbose, but we only care about the end of the output:

`Group count: 
F3D150	5509
Mock	4779
Total of all groups is 10288
Output File Names: 
Data/tutor.trim.contigs.fasta
Data/tutor.trim.contigs.qual
Data/tutor.contigs.report
Data/tutor.scrap.contigs.fasta
Data/tutor.scrap.contigs.qual
Data/tutor.contigs.groups`

We see that there are two groups (F3D150 and Mock) and next to them we see the number of contigs generated per group. Each contig corresponds to - hopefully - a V4 segment of 16S rRNA in the sample.
Notice that a bunch of files have been created in the `Data` folder, which we will use for further processing.
*The workflow with **Mothur** is such that files are produced in each step that will be used in the next step, whenever a file is generated it is listed under `Output File Names` at the end.*

## 2\. Contig analysis and filtering

Let's see some statistics on the contigs! Note, that we do not deal with individual samples, but we process all the contings *together*.

In [24]:
Mothur "summary.seqs(fasta=Data/tutor.trim.contigs.fasta)"

mothur > summary.seqs(fasta=Data/tutor.trim.contigs.fasta)
Using 1 processors.
		Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	249	249	0	3	1
2.5%-tile:	1	252	252	0	3	258
25%-tile:	1	252	252	0	4	2573
Median: 	1	253	253	0	4	5145
75%-tile:	1	253	253	0	5	7717
97.5%-tile:	1	254	254	5	6	10031
Maximum:	1	502	502	249	243	10288
Mean:	1	252.92	252.92	0.639386	4.55978
# of Seqs:	10288
Output File Names: 
Data/tutor.trim.contigs.summary
It took 0 secs to summarize 10288 sequences.


This is an interesting way of showing a histogram of the data. The column names should be self explanatory, except for `Ambigs` (maximum number of ambiguous bases), and `Polymer` (the longest homopolymer) per the category.

We can see that most of the reads are 252-254bp long. We also see that there are contigs with the length of 502, these all contain many ambiguous calls. We can safely discard of the sequences with ambiguity and excessive length, because these arised from incorrect read pairing. To do this we must screen the sequences:

In [27]:
Mothur "screen.seqs(fasta=Data/tutor.trim.contigs.fasta,
                    group=Data/tutor.contigs.groups,
                    maxambig=0,
                    maxlength=255,
                    processors=2)"

mothur > screen.seqs(fasta=Data/tutor.trim.contigs.fasta, group=Data/tutor.contigs.groups, maxambig=0, maxlength=255, processors=2)
Using 2 processors.
Processing sequence: 100
Processing sequence: 200
Processing sequence: 300
Processing sequence: 400
Processing sequence: 500
Processing sequence: 600
Processing sequence: 700
Processing sequence: 800
Processing sequence: 900
Processing sequence: 1000
Processing sequence: 1100
Processing sequence: 1200
Processing sequence: 1300
Processing sequence: 1400
Processing sequence: 1500
Processing sequence: 1600
Processing sequence: 1700
Processing sequence: 1800
Processing sequence: 1900
Processing sequence: 2000
Processing sequence: 2100
Processing sequence: 2200
Processing sequence: 2300
Processing sequence: 2400
Processing sequence: 2500
Processing sequence: 2600
Processing sequence: 2700
Processing sequence: 2800
Processing sequence: 2900
Processing sequence: 3000
Processing sequence: 3100
Processing sequence: 3200
Processing sequence: 3300

Notice the new output file `tutor.trim.contigs.good.fasta` containing the sequences that passed the screen.
We can check if this filtering was successful with sequence statistics again:

In [29]:
Mothur "summary.seqs(fasta=Data/tutor.trim.contigs.good.fasta)"

mothur > summary.seqs(fasta=Data/tutor.trim.contigs.good.fasta)
Using 1 processors.
		Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	251	251	0	3	1
2.5%-tile:	1	252	252	0	3	219
25%-tile:	1	252	252	0	4	2185
Median: 	1	253	253	0	4	4369
75%-tile:	1	253	253	0	5	6553
97.5%-tile:	1	254	254	0	6	8519
Maximum:	1	255	255	0	6	8737
Mean:	1	252.699	252.699	0	4.48083
# of Seqs:	8737
Output File Names: 
Data/tutor.trim.contigs.good.summary
It took 1 secs to summarize 8737 sequences.


The resulting file is free of ambiguities and the contigs have the expected read length.

## 3. Sequence (contig) number reduction

Our files contain duplicates which can be grouped and joined into single groups for faster downstream processing.

In [30]:
Mothur "unique.seqs(fasta= Data/tutor.trim.contigs.good.fasta)"

mothur > unique.seqs(fasta= Data/tutor.trim.contigs.good.fasta)
1000	358
2000	641
3000	893
4000	1121
5000	1324
6000	1458
7000	1582
8000	1685
8737	1774
Output File Names: 
Data/tutor.trim.contigs.good.names
Data/tutor.trim.contigs.good.unique.fasta


We need count the number of occurrences not to loose track of the frequency (weight) of each contig.

In [31]:
Mothur "count.seqs(name=Data/tutor.trim.contigs.good.names,
                   group=Data/tutor.contigs.good.groups)"

mothur > count.seqs(name=Data/tutor.trim.contigs.good.names, group=Data/tutor.contigs.good.groups)
Using 1 processors.
It took 0 secs to create a table for 8737 sequences.
Total number of sequences: 8737
Output File Names: 
Data/tutor.trim.contigs.good.count_table


Now, we can recall the same sequence statistic command on the file with the unique sequences (`tutor.trim.contigs.good.unique.fasta`) and the count table for the occurrence of each contig (`tutor.trim.contigs.good.count_table`) to recover the sequence statistics seen before.

In [34]:
Mothur "summary.seqs(fasta= Data/tutor.trim.contigs.good.unique.fasta,
                     count=Data/tutor.trim.contigs.good.count_table)"

mothur > summary.seqs(fasta= Data/tutor.trim.contigs.good.unique.fasta, count=Data/tutor.trim.contigs.good.count_table)
Using 1 processors.
		Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	251	251	0	3	1
2.5%-tile:	1	252	252	0	3	219
25%-tile:	1	252	252	0	4	2185
Median: 	1	253	253	0	4	4369
75%-tile:	1	253	253	0	5	6553
97.5%-tile:	1	254	254	0	6	8519
Maximum:	1	255	255	0	6	8737
Mean:	1	252.699	252.699	0	4.48083
# of unique seqs:	1774
total # of seqs:	8737
Output File Names: 
Data/tutor.trim.contigs.good.unique.summary
It took 0 secs to summarize 8737 sequences.


At this point we are ready to map the reads onto the reference sequence database.

## 4\. Preparation of the most current Silva reference alignment files for mapping
The goal of this step is to reduce the size of the full alignment flatfile to reduce the memory footprint, and to accelerate the mapping of the contigs onto the sequences within. The file `Silva.nr_v128.tgz` contains two files:

In [35]:
tar tvf Silva.nr_v128.tgz

-rw-r--r-- pschloss/staff 19877941 2017-03-22 08:37 silva.nr_v128.tax
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
-rw-r--r-- pschloss/staff 9554447505 2017-03-22 08:31 silva.nr_v128.align
-rw-r--r-- pschloss/staff      14662 2017-03-22 08:43 README.md


- `silva.nr_v128.tax` : the smaller taxonomy list for the sequences
- `silva.nr_v128.align` : the alignment file (9111.8MB)

We first extract the smaller file.

In [36]:
tar xvfz Silva.nr_v128.tgz silva.nr_v128.tax
mv silva.nr_v128.tax Data/

silva.nr_v128.tax
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'


Then, we reprocess `silva.nr_v128.align` to contain only the V4 region of the 16S rRNA data of the 104,711 unique sequences. According to the tutorial [1] out of the 50,000 column we only need the ones between 11894 and 25319. A blog post [2] explains how to modify the start/end to look for other variable domains (*not recommended*). The gist of it is that this file contains a bunch of rRNA sequences that are aligned *collectively*, but due to the alignemnt they get stretched out with a lot of gaps. Each new iteration of the Silva reference files may or may not move the start/stop sites around, so in order to find an approximate starting point we need to align a reference sequence (e.g. that of *E. coli*) to find the region (V4, V5, V34) of interest.

Here I divert from the procedure suggested by the tutorial. Instead of using **Mothur** to reduce the size of the alignment file I will use a Bash script. The reason is the size of the uncomressed alignment (~8.9GB) which did not fit my Virtual Disk image. I first decompress and recompress the alignment file to get `silva.nr_v128.align.gz` in order to have it separated from the other files in the archive:

In [38]:
tar xvfzO Silva.nr_v128.tgz silva.nr_v128.align | gzip -c - > silva.nr_v128.align.gz
ls -la silva.nr_v128.align.gz

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
silva.nr_v128.align
-rw-rw-r-- 1 viktor viktor 323073676 Mar 23 14:44 silva.nr_v128.align.gz


Note, this produced a much smaller file (311MB).

Next, I will cut out the needed column range and keep the FASTA headers. To keep it economical (small) I use process substitution and combine two streams with `paste`. Even though I *really* wanted, I cannot compress the resulting file for the next step, because - for some mysterious reasons - **Mothur** cannot process the alignment file as a `.align.gz`, even though for some functions (such as `make.contigs()`) using `gzip`ed files is definitely an option.

In [49]:
paste <(gunzip -c silva.nr_v128.align.gz | grep '^>') \
      <(gunzip -c silva.nr_v128.align.gz | grep -v '^>' \
                                         | cut -c 11894-25319 | tr -d '.') \
      -d '\n' > Data/silva.nr_v128.trunc.align
ls -la Data/silva.nr_v128.trunc.align

-rw-rw-r-- 1 viktor viktor 2581212091 Mar 23 15:07 silva.nr_v128.trunc.align


Even this truncated file is 2461.6MB... *(I was thinking about creating a named pipe to the compressed version, but I don't know if **Mothur** needs random access to the file.)*

Moving on. We will first look at some general statistics of the alignment file.

In [52]:
Mothur "summary.seqs(fasta=Data/silva.nr_v128.trunc.align,
                     processors=2)"

mothur > summary.seqs(fasta=Data/silva.nr_v128.trunc.align, processors=2)
Using 2 processors.
mothur > set.logfile(name=mothur.log, append=T)
mothur > summary.seqs(fasta=Data/silva.nr_v128.trunc.align, processors=2)
Using 2 processors.
		Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	9877	219	0	3	1
2.5%-tile:	2	13426	292	0	4	4767
25%-tile:	2	13426	293	0	4	47666
Median: 	2	13426	293	0	5	95331
75%-tile:	2	13426	293	0	5	142996
97.5%-tile:	2	13426	459	1	6	185895
Maximum:	4226	13426	1521	5	16	190661
Mean:	2.27349	13425.8	309.098	0.048164	4.75374
# of Seqs:	190661
Output File Names: 
Data/silva.nr_v128.trunc.summary
It took 27 secs to summarize 190661 sequences.


We can see that we have successfully reduced the 50,000 column wide alignment database which has 190,661 sequences.

## 5\. Align contigs onto the references

This is equivalent of mapping the reads / contigs onto a reference genome, except here we have a lot of references.

In [56]:
Mothur "align.seqs(fasta=Data/tutor.trim.contigs.good.unique.fasta,
                   reference=Data/silva.nr_v128.trunc.align)"

mothur > align.seqs(fasta=Data/tutor.trim.contigs.good.unique.fasta, reference=Data/silva.nr_v128.trunc.align)
Using 1 processors.
Reading in the Data/silva.nr_v128.trunc.align template sequences...	DONE.
It took 89 to read  190661 sequences.
Aligning sequences from Data/tutor.trim.contigs.good.unique.fasta ...
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1774
It took 16 secs to align 1774 sequences.
Output File Names: 
Data/tutor.trim.contigs.good.unique.align
Data/tutor.trim.contigs.good.unique.align.report


Notice that we aligned only 1,774 unique contigs instead of the 8,737 thanks to the deduplication step before.
Let's see some statistics on the aligned sequences. The resulting FASTA file contains the contigs aligned onto the reference with spacing indicated by "`.`" characters, gaps indicated by "`-`" as ususal.

In [12]:
Mothur "summary.seqs(fasta=Data/tutor.trim.contigs.good.unique.align,
                     count=Data/tutor.trim.contigs.good.count_table,
                     processors=2)"

mothur > summary.seqs(fasta=Data/tutor.trim.contigs.good.unique.align, count=Data/tutor.trim.contigs.good.count_table, processors=2)
Using 2 processors.
mothur > set.logfile(name=mothur.log, append=T)
mothur > summary.seqs(fasta=Data/tutor.trim.contigs.good.unique.align, count=Data/tutor.trim.contigs.good.count_table, processors=2)
Using 2 processors.
		Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1969	11550	251	0	3	1
2.5%-tile:	1969	11551	252	0	3	219
25%-tile:	1969	11551	252	0	4	2185
Median: 	1969	11551	253	0	4	4369
75%-tile:	1969	11551	253	0	5	6553
97.5%-tile:	1969	11551	254	0	6	8519
Maximum:	1978	11554	255	0	6	8737
Mean:	1969	11551	252.699	0	4.48083
# of unique seqs:	1774
total # of seqs:	8737
Output File Names: 
Data/tutor.trim.contigs.good.unique.summary
It took 0 secs to summarize 8737 sequences.


This summary shows that the contigs mapped onto more or less the same regions. We do not need additional filtering we can carry on with our analysis. The tutorial [1] has here an additional filtering step to remove the contigs that do not map like the bulk of them.

We know that the aligned contigs mostly overlap, we remove the overhangs (spacer "`.`" to reduce file size).

In [13]:
Mothur "filter.seqs(fasta= Data/tutor.trim.contigs.good.unique.align, vertical=T, trump=.)"

mothur > filter.seqs(fasta= Data/tutor.trim.contigs.good.unique.align, vertical=T, trump=.)
Using 1 processors.
Creating Filter... 
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1774
Running Filter... 
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1774
Length of filtered alignment: 307
Number of columns removed: 13119
Length of the original alignment: 13426
Number of sequences used to construct filter: 1774
Output File Names: 
Data/tutor.filter
Data/tutor.trim.contigs.good.unique.filter.fasta


The resulting file `tutor.trim.contigs.good.unique.filter.fasta` contains the contigs mapped onto a reference genome file, aligned, just like as in an MSA, reduced from 13,426 column alignment to 307 column alignment.

Since we have trimmed the ends of the aligned contigs, some of them may be redundant. Next, we make sure that we only deal with unique contigs:

In [14]:
Mothur "unique.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.fasta,
                    count=Data/tutor.trim.contigs.good.count_table)"

mothur > unique.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.fasta, count=Data/tutor.trim.contigs.good.count_table)
1000	995
1774	1750
Output File Names: 
Data/tutor.trim.contigs.good.unique.filter.count_table
Data/tutor.trim.contigs.good.unique.filter.unique.fasta


In [15]:
Mothur "summary.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.fasta,
                     count=Data/tutor.trim.contigs.good.unique.filter.count_table)"

mothur > summary.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.fasta, count=Data/tutor.trim.contigs.good.unique.filter.count_table)
Using 1 processors.
		Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	307	248	0	3	1
2.5%-tile:	1	307	249	0	3	219
25%-tile:	1	307	249	0	4	2185
Median: 	1	307	250	0	4	4369
75%-tile:	1	307	250	0	5	6553
97.5%-tile:	1	307	251	0	6	8519
Maximum:	1	307	252	0	6	8737
Mean:	1	307	249.701	0	4.48083
# of unique seqs:	1750
total # of seqs:	8737
Output File Names: 
Data/tutor.trim.contigs.good.unique.filter.unique.summary
It took 0 secs to summarize 8737 sequences.


The number of unique sequences has reduced from 1774 to 1750, but these refer to all the contigs that we initially had (8,737).

## 6\. Removing sequencing errors by means of pre-clustering

By preclustering contigs that have 1bp difference per 100bp the sequencing data can be further reduced with minor loss of information. The algorithm sorts the contigs by abundance and merges those that are close to one another and up to 2bp dissimilar.

In [16]:
Mothur "pre.cluster(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.fasta,
                    count=Data/tutor.trim.contigs.good.unique.filter.count_table,
                    diffs=2)"

mothur > pre.cluster(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.fasta, count=Data/tutor.trim.contigs.good.unique.filter.count_table, diffs=2)
Using 1 processors.
Processing group F3D150:
0	1194	48
100	681	561
200	596	646
300	576	666
400	565	677
500	557	685
600	546	696
700	542	700
800	538	704
900	535	707
1000	533	709
1100	531	711
1200	531	711
1242	531	711
Total number of sequences before pre.cluster was 1242.
pre.cluster removed 711 sequences.
It took 0 secs to cluster 1242 sequences.
Processing group Mock:
0	468	54
100	135	387
200	133	389
300	129	393
400	125	397
500	124	398
522	124	398
Total number of sequences before pre.cluster was 522.
pre.cluster removed 398 sequences.
It took 0 secs to cluster 522 sequences.
It took 1 secs to run pre.cluster.
Output File Names: 
Data/tutor.trim.contigs.good.unique.filter.unique.precluster.fasta
Data/tutor.trim.contigs.good.unique.filter.unique.precluster.count_table
Data/tutor.trim.contigs.good.unique.filter.unique.precluster.F3D150.m

This process removed all together 398 + 722 = 1120 contigs. That further reduced the redundancy in our results.

## 7\.Removing chimeras

During the contig assembly there may have been chimeric contigs produced that we want to remove. These are contigs produced by combining the wrong read pairs into a contig. This uses the external tool `vsearch` that we needed to install separately.

In [17]:
Mothur "chimera.vsearch(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.fasta,
                        count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.count_table,
                        dereplicate=T)"

vsearch v2.4.2_linux_x86_64, 3.9GB RAM, 3 cores
https://github.com/torognes/vsearch

Reading file Data/tutor.trim.contigs.good.unique.filter.unique.precluster.temp 100%  
132581 nt in 531 seqs, min 248, max 252, avg 250
Masking 100%  
Sorting by abundance 100%
Counting unique k-mers 100%  
Detecting chimeras 100%  
Found 268 (50.5%) chimeras, 256 (48.2%) non-chimeras,
and 7 (1.3%) borderline sequences in 531 unique sequences.
Taking abundance information into account, this corresponds to
474 (10.3%) chimeras, 4117 (89.4%) non-chimeras,
and 13 (0.3%) borderline sequences in 4604 total sequences.
vsearch v2.4.2_linux_x86_64, 3.9GB RAM, 3 cores
https://github.com/torognes/vsearch

Reading file Data/tutor.trim.contigs.good.unique.filter.unique.precluster.temp 100%  
30988 nt in 124 seqs, min 249, max 252, avg 250
Masking 100% 
Sorting by abundance 100%
Counting unique k-mers 100% 
Detecting chimeras 100%  
Found 62 (50.0%) chimeras, 60 (48.4%) non-chimeras,
and 2 (1.6%) borderline sequence

This function will remove the chimeras from the count file, but not from the FASTA file, we need to do that manually:

In [18]:
Mothur "remove.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.fasta,
                    accnos=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.accnos)"

mothur > remove.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.fasta, accnos=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.accnos)
Removed 330 sequences from your fasta file.
Output File Names: 
Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.fasta


Next we check how much our overall contig database has shrunk by removing some of the unique contigs as chimeras:

In [19]:
Mothur "summary.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.fasta,
                     count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.pick.count_table)"

mothur > summary.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.fasta, count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.pick.count_table)
Using 1 processors.
		Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	307	248	0	3	1
2.5%-tile:	1	307	249	0	3	205
25%-tile:	1	307	249	0	4	2047
Median: 	1	307	250	0	4	4094
75%-tile:	1	307	250	0	5	6140
97.5%-tile:	1	307	251	0	6	7982
Maximum:	1	307	252	0	6	8186
Mean:	1	307	249.715	0	4.48778
# of unique seqs:	312
total # of seqs:	8186
Output File Names: 
Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.summary
It took 0 secs to summarize 8186 sequences.


The reduction is about 6.3% (from 8,737), the tutorial had a 8.1% reduction, this is even less, I believe it is a good sign.

## 8. Classification of contigs into OTUs
Now that we have reduced the number of contigs and removed potential read errors, we can go ahead and classify the sequences using the Bayesian classifier. The reference files for that are the `trainset16_022016.pds.fasta` and the taxonomy information `trainset16_022016.pds.tax`.

In [22]:
Mothur "classify.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.fasta,
                      count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.pick.count_table,
                      reference=PDS/trainset16_022016.pds.fasta, 
                      taxonomy=PDS/trainset16_022016.pds.tax, 
                      cutoff=80)"

mothur > classify.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.fasta, count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.pick.count_table, reference=PDS/trainset16_022016.pds.fasta, taxonomy=PDS/trainset16_022016.pds.tax, cutoff=80)
Using 1 processors.
Generating search database...    DONE.
It took 6 seconds generate search database. 
Reading in the PDS/trainset16_022016.pds.tax taxonomy...	DONE.
Calculating template taxonomy tree...     DONE.
Calculating template probabilities...     DONE.
It took 18 seconds get probabilities. 
Classifying sequences from Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.fasta ...
Processing sequence: 100
Processing sequence: 200
Processing sequence: 300
Processing sequence: 312
It took 5 secs to classify 312 sequences.
It took 0 secs to create the summary file for 312 sequences.
Output File Names: 
Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.pds.wang.t

The data is not accurate enough to pinpoint the actual species with high confidence, this is where we generate the OTUs. We set the OTU clustering to the level of Order (4), to increase the confidence.

In [28]:
Mothur "cluster.split(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.fasta,
                      count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.pick.count_table,
                      taxonomy=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.pds.wang.taxonomy,
                      splitmethod=classify, 
                      taxlevel=4, 

cutoff=0.03)"

mothur > cluster.split(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.fasta, count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.pick.count_table, taxonomy=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.pds.wang.taxonomy, splitmethod=classify, taxlevel=4, cutoff=0.03)
Using 1 processors.
[NOTE]: Default clustering method has changed to opti. To use average neighbor, set method=average.
Splitting the file...
/******************************************/
Running command: dist.seqs(fasta=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.fasta.0.temp, processors=1, cutoff=0.03)
Using 1 processors.
/******************************************/
0	0
100	0
176	0
Output File Names: 
Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.fasta.0.dist
It took 0 seconds to calculate the distances for 177 sequences.
/******************************************/
Running command: dist.seqs(fasta=Data/tut

This function is very verbose, at the end we have list of OTUs, into which each the reads are classified into.

The previous step produced OTU per contig, but it did not care about the sample source. We need to assign these reads to the samples:

In [32]:
Mothur "classify.otu(list=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.opti_mcc.unique_list.list,
                     count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.pick.count_table,
                     taxonomy=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.pds.wang.taxonomy,
                     label=0.03)"

mothur > classify.otu(list=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.opti_mcc.unique_list.list, count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.pick.count_table, taxonomy=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.pds.wang.taxonomy, label=0.03)
0.03	199
Output File Names: 
Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.opti_mcc.unique_list.0.03.cons.taxonomy
Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.opti_mcc.unique_list.0.03.cons.tax.summary


Finally, we obtain the quantitative information: we calculate how many reads belonged into each OTU per sample.

In [30]:
Mothur "make.shared(list=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.opti_mcc.unique_list.list,
                    count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.pick.count_table,
                    label=0.03)"

mothur > make.shared(list=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.opti_mcc.unique_list.list, count=Data/tutor.trim.contigs.good.unique.filter.unique.precluster.denovo.vsearch.pick.count_table, label=0.03)
0.03
Output File Names: 
Data/tutor.trim.contigs.good.unique.filter.unique.precluster.pick.opti_mcc.unique_list.shared


[1] https://www.mothur.org/wiki/MiSeq_SOP

[2] http://blog.mothur.org/2016/07/07/Customization-for-your-region/